# Optimización de Pesos con Datos Humanos (Estrategia 3N - Alineado)

Este notebook optimiza los pesos del modelo basándose en:
1.  **Datos Humanos (N)**: Decisiones reales del profesor cargadas desde un Excel.
2.  **Anclas Sintéticas (2N)**: Estudiantes generados artificialmente ("Ganadores" >3.5, "Perdedores" <2.5).

El modelo de evaluación y la función de pérdida están **alineados** con la versión general (Umbrales Variables + FNR Prioritario).

$$ Loss = MSE_{FN} + MSE_{FP} + FPR + (5.0 \cdot FNR) $$


In [101]:
import numpy as np
import pandas as pd
import re
import copy
from scipy.optimize import minimize, Bounds, LinearConstraint

np.random.seed(42)
pd.set_option('display.float_format', '{:.2f}'.format)

# CONFIGURACIÓN ARCHIVO
EXCEL_FILE = 'encuesta_profesores_20260201_002015.xlsx'


In [102]:
# 1. CONFIGURACIÓN DEL MODELO (Idéntica a General)
config_evaluacion = {
    "regular_items": [
        {"name": "Quiz_1", "weakness": 3.5, "correlated_with": "Tarea_1", "correlation_factor": 0.5, "type": "easy", "threshold": 2.2},
        {"name": "Tarea_1", "weakness": 2.5, "min": 0.05, "max": 0.20, "suggested": 0.15, "type": "advanced", "threshold": 3.0},
        {"name": "Quiz_2", "correlated_with": "Quiz_1", "correlation_factor": 1.2, "weakness": 3.5, "type": "easy", "threshold": 2.5},
        {"name": "Tarea_2", "correlated_with": "Tarea_1", "correlation_factor": 1.2, "weakness": 2.5, "type": "filter", "threshold": 2.8},
        {"name": "Quiz_3", "correlated_with": "Quiz_1", "correlation_factor": 1.0, "weakness": 3.5, "type": "easy", "threshold": 2.0},
        {"name": "Tarea_3", "weakness": 2.5, "correlated_with": "Tarea_1", "correlation_factor": 1.0, "type": "advanced", "threshold": 3.0},
    ],
    "definitory_item": {
        "name": "Examen Final", "sudden_death": 2.50, "max_weight": 0.40, "type": "exam", "threshold": 2.5
    }
}


In [103]:
# 2. CARGAR DATOS HUMANOS (N)
import os

if not os.path.exists(EXCEL_FILE):
    print(f"ERROR: No se encuentra el archivo {EXCEL_FILE}")
else:
    print(f"Cargando {EXCEL_FILE}...")
    df_human_raw = pd.read_excel(EXCEL_FILE)
    
    # Clean Column Names
    clean_cols = {}
    for col in df_human_raw.columns:
        new_col = re.sub(r'\s*\[.*?\]', '', col).strip()
        clean_cols[col] = new_col
        
    df_human = df_human_raw.rename(columns=clean_cols).copy()
    
    # Identify Judgment Column
    judge_col = [c for c in df_human.columns if 'Ganar' in c]
    if not judge_col:
        raise ValueError("No se encontró columna de decisión ('¿Debería Ganar?')")
    judge_col = judge_col[0]
    
    # Filter Valid Judgments (0 or 1)
    df_human = df_human.dropna(subset=[judge_col])
    df_human = df_human[df_human[judge_col].astype(str).str.match(r'^[01](\.0)?$')]
    df_human['y_true'] = df_human[judge_col].astype(int)
    
    print(f"Filas cargadas: {len(df_human_raw)}")
    print(f"Filas con decisión válida (N): {len(df_human)}")
    display(df_human.head())


Cargando encuesta_profesores_20260201_002015.xlsx...
Filas cargadas: 1048575
Filas con decisión válida (N): 100


,Quiz_1,Tarea_1,Quiz_2,Tarea_2,Quiz_3,Tarea_3,Examen Final,"¿Debería Ganar? (1=Sí, 0=No)",Unnamed: 8,Human,Original,Random,Loss,Win,Only exam,y_true
0,3.50,1.90,4.20,4.60,4.00,3.20,3.10,1.00,NaN,1.00,0.00,0,0,1,1,1
1,4.40,3.00,4.40,3.80,4.40,3.70,1.60,0.00,NaN,0.00,0.00,0,0,1,0,0
2,4.20,4.00,3.80,0.80,4.30,2.80,3.20,0.00,NaN,0.00,1.00,0,0,1,1,0
3,2.40,3.00,4.10,1.30,3.80,2.30,5.00,0.00,NaN,0.00,1.00,1,0,1,1,0
4,3.60,4.40,4.50,1.70,4.00,2.40,3.80,1.00,NaN,1.00,1.00,1,0,1,1,1


In [104]:
# 3. GENERADOR SINTÉTICO (Para Anclas)
def generar_distribucion_bimodal(n, low_center=1.5, high_center=4.5, ratio=0.5, sigma=0.6):
    n_high = int(n * ratio)
    n_low = n - n_high
    h = np.random.normal(high_center, sigma, n_high)
    l = np.random.normal(low_center, sigma, n_low)
    return np.clip(np.concatenate([h, l]), 0, 5)

def generate_data(config, N=2000):
    data = {}
    
    def get_col(dtype):
        if dtype == 'filter': return generar_distribucion_bimodal(N, 1.2, 4.0, 0.4)
        elif dtype == 'easy': return np.clip(4.5 - np.random.exponential(0.8, N), 0, 5)
        elif dtype == 'exam': return np.clip(np.random.normal(2.8, 1.1, N), 0, 5)
        else: return np.clip(np.random.normal(3.0, 1.0, N), 0, 5)

    for item in config['regular_items']:
        data[item['name']] = get_col(item.get('type', 'advanced'))
    def_item = config['definitory_item']
    data[def_item['name']] = get_col(def_item.get('type', 'advanced'))
    
    df = pd.DataFrame(data).sample(frac=1).reset_index(drop=True).round(2)
    return df

# Derived Weights Helper
def get_derived_weights(config_in):
    reg_items = config_in['regular_items']
    def_item = config_in['definitory_item']
    weight_map = {}
    
    for item in reg_items:
        if 'weight' in item: weight_map[item['name']] = item['weight']
        elif 'correlated_with' not in item: weight_map[item['name']] = item.get('peso_sugerido', 0.15)

    for _ in range(3):
        for item in reg_items:
             w = 0.0
             if 'weight' in item: w = item['weight']
             elif 'correlated_with' in item:
                 p = item['correlated_with']
                 if p in weight_map: w = weight_map[p] * item['correlation_factor']
             else: w = item.get('peso_sugerido', 0.15)
             weight_map[item['name']] = w
             
    final_reg_weights = np.array([weight_map[item['name']] for item in reg_items])
    if 'weight' in def_item: ft_w = def_item['weight']
    else: ft_w = max(0.0, 1.0 - np.sum(final_reg_weights))
        
    return final_reg_weights, ft_w

reg_weights_sugg, w_final_sugg = get_derived_weights(config_evaluacion)
full_weights_sugg = np.concatenate([reg_weights_sugg, [w_final_sugg]])


In [105]:
# 4. GENERACIÓN DE ANCLAS (2N)

N_human = len(df_human)
if N_human == 0:
    print("WARNING: No hay datos humanos. Usando N=50 para demo.")
    N_human = 50

# Generate Pool
df_pool = generate_data(config_evaluacion, N=N_human * 20) 

# Calculate Average (Suggested)
reg_names = [i['name'] for i in config_evaluacion['regular_items']]
def_name = config_evaluacion['definitory_item']['name']
cols_score = reg_names + [def_name]

scores = df_pool[cols_score].values
avg_projected = np.dot(scores, full_weights_sugg)

# High Anchors (>3.5 -> 1)
mask_high = avg_projected > 3.5
df_high = df_pool[mask_high].sample(n=min(sum(mask_high), N_human), replace=False).copy()
df_high['y_true'] = 1

# Low Anchors (<2.5 -> 0)
mask_low = avg_projected < 2.5
df_low = df_pool[mask_low].sample(n=min(sum(mask_low), N_human), replace=False).copy()
df_low['y_true'] = 0

print(f"Anclas Generadas: {len(df_high)} (High) + {len(df_low)} (Low)")


Anclas Generadas: 100 (High) + 100 (Low)


In [106]:
# 5. DATASET DE ENTRENAMIENTO (3N)
cols_keep = reg_names + [def_name] + ['y_true']

df_train = pd.concat([
    df_human[cols_keep],
    df_high[cols_keep],
    df_low[cols_keep]
], axis=0).reset_index(drop=True)

print(f"Dataset Final: {len(df_train)} filas.")
print(df_train['y_true'].value_counts())
display(df_train.head())


Dataset Final: 300 filas.
y_true
0    153
1    147
Name: count, dtype: int64


,Quiz_1,Tarea_1,Quiz_2,Tarea_2,Quiz_3,Tarea_3,Examen Final,y_true
0,3.50,1.90,4.20,4.60,4.00,3.20,3.10,1
1,4.40,3.00,4.40,3.80,4.40,3.70,1.60,0
2,4.20,4.00,3.80,0.80,4.30,2.80,3.20,0
3,2.40,3.00,4.10,1.30,3.80,2.30,5.00,0
4,3.60,4.40,4.50,1.70,4.00,2.40,3.80,1


In [107]:
# 6. LOGICA DE EVALUACIÓN (Alineada con General)
# Usa Thresholds + U-Factors + Regla de 3.0

def calculate_grade_strict_general(weights_regular, df, config):
    reg_items = config['regular_items']
    def_item = config['definitory_item']
    
    w_sum_reg = np.sum(weights_regular)
    w_def = 1.0 - w_sum_reg
    all_weights = np.concatenate([weights_regular, [w_def]])
    
    reg_names = [item['name'] for item in reg_items]
    def_name = def_item['name']
    all_names = reg_names + [def_name]
    
    grades_matrix = df[all_names].values
    
    # Extract Thresholds Vector
    thresholds = []
    for item in reg_items:
        thresholds.append(item.get('threshold', 2.5))
    thresholds.append(def_item.get('threshold', 2.5))
    T_vector = np.array(thresholds)
    
    # U-Factors
    ratios = np.clip(grades_matrix / T_vector, 0, 1)
    n_items = len(all_names)
    u_factors = np.zeros_like(grades_matrix)
    
    for i in range(n_items):
        mask = np.ones(n_items, dtype=bool)
        mask[i] = False
        u_factors[:, i] = np.prod(ratios[:, mask], axis=1)
        
    # Grades
    weighted_grades_u = grades_matrix * all_weights * u_factors
    final_strict = np.sum(weighted_grades_u, axis=1)
    
    weighted_grades_raw = grades_matrix * all_weights
    final_raw = np.sum(weighted_grades_raw, axis=1)
    
    # Strict Fail Rule (< 3.0 -> 0.8 * Raw)
    mask_fail = final_raw < 3.0
    final_grade = np.where(mask_fail, 0.8 * final_raw, final_strict)
    
    return final_grade


In [108]:
# 7. OPTIMIZACIÓN (Objetivo WEIGHTED FNR)

# Helpers for Indep Weights
def get_indep_indices(config):
    return [i for i, item in enumerate(config['regular_items']) if 'correlated_with' not in item]

def reconstruct_full_weights_indep(x_independent, config):
    items = config['regular_items']
    indep_indices = get_indep_indices(config)
    
    weight_map = {}
    full_weights = np.zeros(len(items))
    
    # Fill Indep
    for k, idx in enumerate(indep_indices):
        val = x_independent[k]
        weight_map[items[idx]['name']] = val
        full_weights[idx] = val
        
    # Fill Dep
    for _ in range(3):
        for i, item in enumerate(items):
            if full_weights[i] == 0.0 and 'correlated_with' in item:
                parent = item['correlated_with']
                if parent in weight_map:
                    val = weight_map[parent] * item['correlation_factor']
                    full_weights[i] = val
                    weight_map[item['name']] = val
    return full_weights

def objective_function(x_independent, df, config):
    weights_regular = reconstruct_full_weights_indep(x_independent, config)
    
    # TARGET: Fixed y_true (Human/Anchors)
    y_true = df['y_true'].values
    y_pred_grade = calculate_grade_strict_general(weights_regular, df, config)
    
    pass_th = 2.95
    N = len(y_true)
    
    # 1. Ajuste (MSE)
    err_fn_sq = np.maximum(0, pass_th - y_pred_grade[y_true==1])**2
    err_fp_sq = np.maximum(0, y_pred_grade[y_true==0] - pass_th)**2
    mse_fn = np.sum(err_fn_sq) / N
    mse_fp = np.sum(err_fp_sq) / N
    
    # 2. Métricas
    passed = y_pred_grade >= 2.95
    
    fp_count = ((y_true == 0) & passed).sum()
    tn_count = ((y_true == 0) & ~passed).sum()
    fpr = fp_count / (fp_count + tn_count + 1e-9)
    
    fn_count = ((y_true == 1) & ~passed).sum()
    tp_count = ((y_true == 1) & passed).sum()
    fnr = fn_count / (fn_count + tp_count + 1e-9)
    
    # 3. LOSS (Weighted FNR)
    LAMBDA_FNR = 5.0
    total_loss = mse_fn + mse_fp + fpr + (LAMBDA_FNR * fnr)
    return total_loss

# Setup
indep_idx = get_indep_indices(config_evaluacion)
x0 = np.array([config_evaluacion['regular_items'][i]['suggested'] for i in indep_idx])
bounds = Bounds(
    [config_evaluacion['regular_items'][i]['min'] for i in indep_idx],
    [config_evaluacion['regular_items'][i]['max'] for i in indep_idx]
)

# Constraints
coeffs = np.zeros(len(indep_idx))
name_to_k = {config_evaluacion['regular_items'][i]['name']: k for k, i in enumerate(indep_idx)}

for i, item in enumerate(config_evaluacion['regular_items']):
    if 'correlated_with' in item:
        p = item['correlated_with']
        f = item['correlation_factor']
        if p in name_to_k: coeffs[name_to_k[p]] += f
    else:
        if item['name'] in name_to_k: coeffs[name_to_k[item['name']]] += 1.0

def_item = config_evaluacion['definitory_item']
min_w_reg = 1.0 - def_item['max_weight']
linear_constraint = LinearConstraint(coeffs, min_w_reg, 0.99)

print("Optimizando (Prioridad FNR x5.0) sobre datos 3N...")
res = minimize(
    objective_function, x0, 
    args=(df_train, config_evaluacion),
    method='trust-constr', bounds=bounds, constraints=[linear_constraint],
    options={'verbose': 1}
)

print(f"\nExitosa: {res.success}")
opt_indep = res.x
opt_weights_full = reconstruct_full_weights_indep(opt_indep, config_evaluacion)
opt_final_weight = 1.0 - np.sum(opt_weights_full)


Optimizando (Prioridad FNR x5.0) sobre datos 3N...
`xtol` termination condition is satisfied.
Number of iterations: 151, function evaluations: 224, CG iterations: 139, optimality: 1.40e-05, constraint violation: 0.00e+00, execution time: 0.12 s.

Exitosa: True


/Users/jzuluaga/dev/docente/.docente/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py:385: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


In [109]:
# 8. COMPARATIVA FINAL (Alineada)

# A. Crear configuración optimizada
config_opt = copy.deepcopy(config_evaluacion)
for i, item in enumerate(config_opt['regular_items']):
    item['weight'] = float(opt_weights_full[i])
config_opt['definitory_item']['weight'] = float(opt_final_weight)

# B. Rutina de Reporte
def calculate_metrics_report(config_in, df, title="Config"):
    w_reg, w_def = get_derived_weights(config_in)
    
    y_true = df['y_true'].values
    grade_final = calculate_grade_strict_general(w_reg, df, config_in)
    
    pass_th = 2.95
    N = len(y_true)
    
    # MSE
    err_fn_sq = np.maximum(0, pass_th - grade_final[y_true==1])**2
    err_fp_sq = np.maximum(0, grade_final[y_true==0] - pass_th)**2
    mse_fn = np.sum(err_fn_sq) / N
    mse_fp = np.sum(err_fp_sq) / N
    
    # Classif
    passed = grade_final >= 2.95
    fp = ((y_true == 0) & passed).sum()
    tn = ((y_true == 0) & ~passed).sum()
    fpr = fp / (fp + tn + 1e-9)
    
    fn = ((y_true == 1) & ~passed).sum()
    tp = ((y_true == 1) & passed).sum()
    fnr = fn / (fn + tp + 1e-9)
    
    LAMBDA_FNR = 5.0
    loss = mse_fn + mse_fp + fpr + (LAMBDA_FNR * fnr)
    
    print(f"[{title:20}] Loss(Wt): {loss:6.4f} | FPR: {fpr:6.2%} | FNR: {fnr:6.2%} | MSE: {(mse_fn+mse_fp):6.4f}")

print("=== COMPARATIVA DE MÉTRICAS (Weighted FNR x5.0) ===")
calculate_metrics_report(config_evaluacion, df_train, title="ORIGINAL (Sugerido)")
calculate_metrics_report(config_opt, df_train, title="OPTIMIZADO (Human)")

print(f"\n{'ITEM':<15} | {'ORIGINAL':<10} | {'OPTIMIZADO':<10}")
print("-" * 45)
for i, item in enumerate(config_evaluacion['regular_items']):
    w_orig = full_weights_sugg[i]
    w_opt = opt_weights_full[i]
    print(f"{item['name']:<15} | {w_orig:<10.2%} | {w_opt:<10.2%}")
print("-" * 45)
print(f"{'Examen Final':<15} | {w_final_sugg:<10.2%} | {opt_final_weight:<10.2%}")


=== COMPARATIVA DE MÉTRICAS (Weighted FNR x5.0) ===
[ORIGINAL (Sugerido) ] Loss(Wt): 2.9430 | FPR:  1.31% | FNR: 51.02% | MSE: 0.3789
[OPTIMIZADO (Human)  ] Loss(Wt): 2.9122 | FPR:  1.31% | FNR: 51.70% | MSE: 0.3141

ITEM            | ORIGINAL   | OPTIMIZADO
---------------------------------------------
Quiz_1          | 7.50%      | 9.40%     
Tarea_1         | 15.00%     | 18.79%    
Quiz_2          | 9.00%      | 11.28%    
Tarea_2         | 18.00%     | 22.55%    
Quiz_3          | 7.50%      | 9.40%     
Tarea_3         | 15.00%     | 18.79%    
---------------------------------------------
Examen Final    | 28.00%     | 9.79%     
